In [1]:
print("xxx_xyz_ext_pos.ipynb")

xxx_xyz_ext_pos.ipynb


In [2]:
# Libraries
import sys
sys.path.append("../2_depend") # go to parent dir
from libsy import *

In [3]:
# Make folder for output
pathlib.Path('../6_int/confout/').mkdir(parents=True, exist_ok=True)

In [4]:
# Run analysis for each movie (.mat file)
start1 = time.time()
fileNames = glob.glob("../3_input/confy/*conf.mat")
fileNames = sorted(fileNames)

for f in fileNames:
    #
    # Select file
    #f = '../3_input/confy/HET5WT2_t0_conf.mat'
    fileName = f
    # Experiment info
    nm = fileName
    nm = nm.replace("../3_input/confy/","")
    nm = nm.replace(".mat","")
    info = nm
    info = nm.split("_")
    thing = info[0]
    t = info[1]
    confirmed = info[2]
    strain = thing[0:4]
    cond = thing[4:6]
    exp = thing[6:7]
    #
    #
    # Load positions
    data = scipy.io.loadmat(fileName)
    print("    in: "+fileName)
    #
    #
    ### get the number of timepoints in timecourse
    celltotal = len(data['pos'])
    maxtpvec = []
    maxtpvec.append(0)
    #
    for i in range(celltotal):
        #i = 8
        cell = i
        hey = pd.DataFrame(data['pos'][cell][0])
        if np.nansum(hey[0].values) == 0:
            empty = "yes"
        elif np.nansum(hey[0].values) != 0:
            empty = "no"
        if empty == "no":
            times = hey
            maxtp = np.nanmax(hey)
            maxtpvec.append(maxtp)
    #
    maxofmax = np.nanmax(maxtpvec)
    nmax = int(maxofmax)
    #
    #
    # Run for each cell
    for i in range(celltotal):

        # Get dots from each cell
        #i = 23 # +1 in R
        cellno = i
        cellpos = pd.DataFrame(data['pos'][cellno][0])
        dottp = cellpos[0]

        # detect nan and duplicated time points
        na = np.isnan(cellpos[0])
        nna = [] 
        nn = []
        for n in na:
            n = str(n)
            if n == "False":
                nn = "FALSE"
            if n =="True":
                nn = "TRUE"
            nna.append(nn)
        dup = cellpos[0].duplicated(keep='first')
        dup = np.where(dup==False, 'FALSE', 'TRUE')
        dupdf = pd.DataFrame({"tp":dottp, "na": nna, "duplicate": dup})
        dupdf['notnum'] = np.where(dupdf['na']=='FALSE', 0, 1)
        dupdf['dupnum'] = np.where(dupdf['duplicate']=='FALSE', 0, 1)
        mdk = dupdf

        # number the sequence
        x1n = cellpos[1]
        y1n = cellpos[2]
        z1n = cellpos[3]
        seqy = []
        for r in range(1,len(mdk['tp'])+1):
            seqy.append(r)
        seqy = np.array(seqy)
        dee = pd.DataFrame({"seqy":seqy, "mdktp":mdk['tp'].values})
        dee['x1n'] = x1n
        dee['y1n'] = y1n
        dee['z1n'] = z1n

        # remove timepoints with too many spots
        toomany = mdk.groupby('tp')['dupnum'].agg(['sum'])
        toomany = toomany.reset_index()
        toomany = toomany.rename(columns={"sum": "numby"})
        toomanytp = toomany[toomany['numby'] > 1]['tp'].values
        if len(toomanytp) == 0:
            dee = dee
            mdk = mdk
        else:
            dee = dee[-dee['mdktp'].isin(toomanytp)]
            mdk = mdk[-mdk['tp'].isin(toomanytp)]

        #
        #

        ## Spottr begin
        # if spottr is 0, there is not spots to analyze
        spottr = np.nansum(dee['mdktp'])
        if spottr==0:
            "sr"
        else:
            # Prepare variables
            freqqy = mdk['tp'].value_counts().reset_index()
            freqqy = freqqy.sort_values(by=['index'])
            freqqy = freqqy.rename(columns={"index": "Var1", "tp":"Freq"})
            freqqy['Var1'] = freqqy['Var1'].apply(int)
            sumf = np.sum(freqqy['Freq'])
            sumv = len(freqqy['Freq'])
            sumv2v = len(freqqy['Freq'])*2
            shuu = ""
            eg2011cleaned = []
            #
            #
            # PAIRED list
            # if sumf == sumv2v, the cell is unpaired for the whole timecourse
            if sumv2v==sumf:
                shuu = "u"
            else:
                paired = freqqy[freqqy['Freq'] == 1]
                pairlst = paired['Var1'].values
                # if pairlst is empty, there is no paired tp
                # if pairlst has values, create empty second xyz point
                if len(pairlst)==0:
                    "nn"
                else:
                    eg2011cleaned = dee[dee['mdktp'].isin(pairlst)]
                    eg2011cleaned['comment'] = "paired"
                    eg2011cleaned['X2'] = "-"
                    eg2011cleaned['Y2'] = "-"
                    eg2011cleaned['Z2'] = "-"
                    eg2011cleaned['com'] = "pair"
                    qq = eg2011cleaned
                    pimmy = pd.DataFrame({
                        "M":qq['mdktp'], 
                        "X1":qq['x1n'], "Y1":qq['y1n'], "Z1":qq['z1n'], 
                        "X2":qq['X2'], "Y2":qq['Y2'], "Z2":qq['Z2'],
                        "com":qq['com']})
                    pimmy = pimmy[['M', 'X1', 'Y1', 'Z1', 'X2', 'Y2', 'Z2', 'com']]
            #
            #
            # UNPAIRED list
            # if sumv==sumf, the cell is paired for the whole timecourse
            if sumv==sumf:
                shuu = "p"
            else:
                unpaired = freqqy[freqqy['Freq']==2]
                unpairlst = unpaired['Var1'].values
                cleanedunp = dee[dee['mdktp'].isin(unpairlst)]
                unpcl = cleanedunp.drop(columns=['seqy'])
                ssq = [1,2]
                ll = len(unpcl['mdktp'])
                oddy = np.resize(ssq, int(ll))
                unpcl['oddy'] = oddy
                unpcl = unpcl.reset_index()
                unpcl = unpcl.drop(columns=['index'])
                p1 = unpcl[unpcl['oddy']==1]
                p2 = unpcl[unpcl['oddy']==2]
                p1 = p1.reset_index()
                p1 = p1.drop(columns=['index'])
                p2 = p2.reset_index()
                p2 = p2.drop(columns=['index'])
                ghy = pd.DataFrame({
                    "M":p1['mdktp'], 
                    "X1":p1['x1n'], "Y1":p1['y1n'], "Z1":p1['z1n'], 
                    "X2":p2['x1n'], "Y2":p2['y1n'], "Z2":p2['z1n']})
                ghy = ghy[['M', 'X1', 'Y1', 'Z1', 'X2', 'Y2', 'Z2']]
                ghy['com'] = 'unp'
            # if cell is unpaired, take unpaired values
            # if cell is paired, take paired values
            # if cell is mix, combine unpaired and paired values
            if shuu == "u":
                unppai = ghy
            else:
                if shuu == "p":
                    unppai = pimmy
                else:
                    unppai = pd.concat([ghy, pimmy])
            # reset cell variable
            shuu=""
            # main SpottR end

        #
        #

        # Detect missing timepoints
        chronoseq = []
        for n in range(1,nmax+1):
            chronoseq.append(n)
        chronoseq = np.array(chronoseq)
        missing = np.setdiff1d(chronoseq,mdk['tp'].unique())
        # Create empty values for missing timepoints
        if len(missing) == 0:
            mzz = ""
        else:
            mzz = pd.DataFrame({"M":missing,
                               "X1":"n", "Y1":"n", "Z1":"n",
                               "X2":"n", "Y2":"n", "Z2":"n",
                               "com":"NA"})
            mzz = mzz[['M', 'X1', 'Y1', 'Z1', 'X2', 'Y2', 'Z2', 'com']]
        # Combine any paired, unpaired, and missing values
        if spottr==0:
            unppaina = mzz
        else:
            if len(missing) == 0:
                unppaina = unppai
            else:
                unppaina = pd.concat([unppai, mzz])
        unppaina = unppaina.sort_values(by=['M'])
        unppainam = unppaina[unppaina['M']!=""]

        #
        #

        # Prepare table for export
        posdf = unppainam
        postable = posdf.sort_values(by=['M', 'com'])
        postablechrons = chronoseq * 30
        postable['strain'] = strain
        postable['cell'] = cellno+1
        postable['exp'] = exp
        postable['x1'] = "" # Redundant
        postable['y1'] = "" # Redundant
        postable['z1'] = "" # Redundant
        postable['x2'] = "" # Redundant
        postable['y2'] = "" # Redundant
        postable['z2'] = "" # Redundant
        postable['meiosis'] = t
        postable['tpn'] = postable['M']
        postable['M'] = postable['M'].apply(int)
        postable['tps'] = (postable['M']-1)*30
        postable['cond'] = cond
        ptab = postable
        ptabcol = ptab[['strain', 'cond', 'exp', 'meiosis', 'cell', 'tpn', 'tps', 'com',
                       'X1', 'Y1', 'Z1', 'X2', 'Y2', 'Z2']]
        ptabcol.to_csv("../6_int/confout/"+"pos_"+str(nm)+"_cell"+str(cellno+1).zfill(3)+".csv", sep=',', index=False)
        print("    out: "+"../6_int/confout/"+"pos_"+str(nm)+"_cell"+str(cellno+1).zfill(3)+".csv")
        unppainam = []
        # End of cell for loop
    # End of filename for loop
end1 = time.time()
print("time:")
print(end1 - start1) 

    in: ../3_input/confy/HET5WT2_t0_conf.mat
    out: ../6_int/confout/pos_HET5WT2_t0_conf_cell001.csv
    out: ../6_int/confout/pos_HET5WT2_t0_conf_cell002.csv
    out: ../6_int/confout/pos_HET5WT2_t0_conf_cell003.csv
    out: ../6_int/confout/pos_HET5WT2_t0_conf_cell004.csv
    out: ../6_int/confout/pos_HET5WT2_t0_conf_cell005.csv
    out: ../6_int/confout/pos_HET5WT2_t0_conf_cell006.csv
    out: ../6_int/confout/pos_HET5WT2_t0_conf_cell007.csv
    out: ../6_int/confout/pos_HET5WT2_t0_conf_cell008.csv
    out: ../6_int/confout/pos_HET5WT2_t0_conf_cell009.csv
    out: ../6_int/confout/pos_HET5WT2_t0_conf_cell010.csv
    out: ../6_int/confout/pos_HET5WT2_t0_conf_cell011.csv
    out: ../6_int/confout/pos_HET5WT2_t0_conf_cell012.csv
    out: ../6_int/confout/pos_HET5WT2_t0_conf_cell013.csv
    out: ../6_int/confout/pos_HET5WT2_t0_conf_cell014.csv
    out: ../6_int/confout/pos_HET5WT2_t0_conf_cell015.csv
    out: ../6_int/confout/pos_HET5WT2_t0_conf_cell016.csv
    out: ../6_int/confout/p

    out: ../6_int/confout/pos_HET5WT2_t1_conf_cell040.csv
    out: ../6_int/confout/pos_HET5WT2_t1_conf_cell041.csv
    out: ../6_int/confout/pos_HET5WT2_t1_conf_cell042.csv
    out: ../6_int/confout/pos_HET5WT2_t1_conf_cell043.csv
    out: ../6_int/confout/pos_HET5WT2_t1_conf_cell044.csv
    out: ../6_int/confout/pos_HET5WT2_t1_conf_cell045.csv
    out: ../6_int/confout/pos_HET5WT2_t1_conf_cell046.csv
    out: ../6_int/confout/pos_HET5WT2_t1_conf_cell047.csv
    out: ../6_int/confout/pos_HET5WT2_t1_conf_cell048.csv
    out: ../6_int/confout/pos_HET5WT2_t1_conf_cell049.csv
    out: ../6_int/confout/pos_HET5WT2_t1_conf_cell050.csv
    out: ../6_int/confout/pos_HET5WT2_t1_conf_cell051.csv
    out: ../6_int/confout/pos_HET5WT2_t1_conf_cell052.csv
    out: ../6_int/confout/pos_HET5WT2_t1_conf_cell053.csv
    out: ../6_int/confout/pos_HET5WT2_t1_conf_cell054.csv
    out: ../6_int/confout/pos_HET5WT2_t1_conf_cell055.csv
    out: ../6_int/confout/pos_HET5WT2_t1_conf_cell056.csv
    out: ../6_

    out: ../6_int/confout/pos_HET5WT2_t3_conf_cell012.csv
    out: ../6_int/confout/pos_HET5WT2_t3_conf_cell013.csv
    out: ../6_int/confout/pos_HET5WT2_t3_conf_cell014.csv
    out: ../6_int/confout/pos_HET5WT2_t3_conf_cell015.csv
    out: ../6_int/confout/pos_HET5WT2_t3_conf_cell016.csv
    out: ../6_int/confout/pos_HET5WT2_t3_conf_cell017.csv
    out: ../6_int/confout/pos_HET5WT2_t3_conf_cell018.csv
    out: ../6_int/confout/pos_HET5WT2_t3_conf_cell019.csv
    out: ../6_int/confout/pos_HET5WT2_t3_conf_cell020.csv
    out: ../6_int/confout/pos_HET5WT2_t3_conf_cell021.csv
    out: ../6_int/confout/pos_HET5WT2_t3_conf_cell022.csv
    out: ../6_int/confout/pos_HET5WT2_t3_conf_cell023.csv
    out: ../6_int/confout/pos_HET5WT2_t3_conf_cell024.csv
    out: ../6_int/confout/pos_HET5WT2_t3_conf_cell025.csv
    out: ../6_int/confout/pos_HET5WT2_t3_conf_cell026.csv
    out: ../6_int/confout/pos_HET5WT2_t3_conf_cell027.csv
    out: ../6_int/confout/pos_HET5WT2_t3_conf_cell028.csv
    out: ../6_

    out: ../6_int/confout/pos_HET5WT2_t5_conf_cell010.csv
    out: ../6_int/confout/pos_HET5WT2_t5_conf_cell011.csv
    out: ../6_int/confout/pos_HET5WT2_t5_conf_cell012.csv
    out: ../6_int/confout/pos_HET5WT2_t5_conf_cell013.csv
    out: ../6_int/confout/pos_HET5WT2_t5_conf_cell014.csv
    out: ../6_int/confout/pos_HET5WT2_t5_conf_cell015.csv
    out: ../6_int/confout/pos_HET5WT2_t5_conf_cell016.csv
    out: ../6_int/confout/pos_HET5WT2_t5_conf_cell017.csv
    out: ../6_int/confout/pos_HET5WT2_t5_conf_cell018.csv
    out: ../6_int/confout/pos_HET5WT2_t5_conf_cell019.csv
    out: ../6_int/confout/pos_HET5WT2_t5_conf_cell020.csv
    out: ../6_int/confout/pos_HET5WT2_t5_conf_cell021.csv
    out: ../6_int/confout/pos_HET5WT2_t5_conf_cell022.csv
    out: ../6_int/confout/pos_HET5WT2_t5_conf_cell023.csv
    out: ../6_int/confout/pos_HET5WT2_t5_conf_cell024.csv
    out: ../6_int/confout/pos_HET5WT2_t5_conf_cell025.csv
    out: ../6_int/confout/pos_HET5WT2_t5_conf_cell026.csv
    out: ../6_

    out: ../6_int/confout/pos_HET5WT3_t0_conf_cell069.csv
    out: ../6_int/confout/pos_HET5WT3_t0_conf_cell070.csv
    out: ../6_int/confout/pos_HET5WT3_t0_conf_cell071.csv
    out: ../6_int/confout/pos_HET5WT3_t0_conf_cell072.csv
    out: ../6_int/confout/pos_HET5WT3_t0_conf_cell073.csv
    out: ../6_int/confout/pos_HET5WT3_t0_conf_cell074.csv
    out: ../6_int/confout/pos_HET5WT3_t0_conf_cell075.csv
    out: ../6_int/confout/pos_HET5WT3_t0_conf_cell076.csv
    out: ../6_int/confout/pos_HET5WT3_t0_conf_cell077.csv
    out: ../6_int/confout/pos_HET5WT3_t0_conf_cell078.csv
    out: ../6_int/confout/pos_HET5WT3_t0_conf_cell079.csv
    out: ../6_int/confout/pos_HET5WT3_t0_conf_cell080.csv
    out: ../6_int/confout/pos_HET5WT3_t0_conf_cell081.csv
    out: ../6_int/confout/pos_HET5WT3_t0_conf_cell082.csv
    out: ../6_int/confout/pos_HET5WT3_t0_conf_cell083.csv
    out: ../6_int/confout/pos_HET5WT3_t0_conf_cell084.csv
    out: ../6_int/confout/pos_HET5WT3_t0_conf_cell085.csv
    out: ../6_

...

    out: ../6_int/confout/pos_URA3WT8_ta_conf_cell078.csv
    out: ../6_int/confout/pos_URA3WT8_ta_conf_cell079.csv
    out: ../6_int/confout/pos_URA3WT8_ta_conf_cell080.csv
    out: ../6_int/confout/pos_URA3WT8_ta_conf_cell081.csv
    out: ../6_int/confout/pos_URA3WT8_ta_conf_cell082.csv
    out: ../6_int/confout/pos_URA3WT8_ta_conf_cell083.csv
    out: ../6_int/confout/pos_URA3WT8_ta_conf_cell084.csv
    out: ../6_int/confout/pos_URA3WT8_ta_conf_cell085.csv
    out: ../6_int/confout/pos_URA3WT8_ta_conf_cell086.csv
    out: ../6_int/confout/pos_URA3WT8_ta_conf_cell087.csv
    out: ../6_int/confout/pos_URA3WT8_ta_conf_cell088.csv
    out: ../6_int/confout/pos_URA3WT8_ta_conf_cell089.csv
    out: ../6_int/confout/pos_URA3WT8_ta_conf_cell090.csv
    out: ../6_int/confout/pos_URA3WT8_ta_conf_cell091.csv
    in: ../3_input/confy/URA3WT9_t0_conf.mat
    out: ../6_int/confout/pos_URA3WT9_t0_conf_cell001.csv
    out: ../6_int/confout/pos_URA3WT9_t0_conf_cell002.csv
    out: ../6_int/confout/p

    out: ../6_int/confout/pos_URA3WT9_t0_conf_cell130.csv
    out: ../6_int/confout/pos_URA3WT9_t0_conf_cell131.csv
    out: ../6_int/confout/pos_URA3WT9_t0_conf_cell132.csv
    in: ../3_input/confy/URA3WT9_t1_conf.mat
    out: ../6_int/confout/pos_URA3WT9_t1_conf_cell001.csv
    out: ../6_int/confout/pos_URA3WT9_t1_conf_cell002.csv
    out: ../6_int/confout/pos_URA3WT9_t1_conf_cell003.csv
    out: ../6_int/confout/pos_URA3WT9_t1_conf_cell004.csv
    out: ../6_int/confout/pos_URA3WT9_t1_conf_cell005.csv
    out: ../6_int/confout/pos_URA3WT9_t1_conf_cell006.csv
    out: ../6_int/confout/pos_URA3WT9_t1_conf_cell007.csv
    out: ../6_int/confout/pos_URA3WT9_t1_conf_cell008.csv
    out: ../6_int/confout/pos_URA3WT9_t1_conf_cell009.csv
    out: ../6_int/confout/pos_URA3WT9_t1_conf_cell010.csv
    out: ../6_int/confout/pos_URA3WT9_t1_conf_cell011.csv
    out: ../6_int/confout/pos_URA3WT9_t1_conf_cell012.csv
    out: ../6_int/confout/pos_URA3WT9_t1_conf_cell013.csv
    out: ../6_int/confout/p

    out: ../6_int/confout/pos_URA3WT9_t2_conf_cell046.csv
    out: ../6_int/confout/pos_URA3WT9_t2_conf_cell047.csv
    out: ../6_int/confout/pos_URA3WT9_t2_conf_cell048.csv
    out: ../6_int/confout/pos_URA3WT9_t2_conf_cell049.csv
    out: ../6_int/confout/pos_URA3WT9_t2_conf_cell050.csv
    out: ../6_int/confout/pos_URA3WT9_t2_conf_cell051.csv
    out: ../6_int/confout/pos_URA3WT9_t2_conf_cell052.csv
    out: ../6_int/confout/pos_URA3WT9_t2_conf_cell053.csv
    out: ../6_int/confout/pos_URA3WT9_t2_conf_cell054.csv
    out: ../6_int/confout/pos_URA3WT9_t2_conf_cell055.csv
    out: ../6_int/confout/pos_URA3WT9_t2_conf_cell056.csv
    out: ../6_int/confout/pos_URA3WT9_t2_conf_cell057.csv
    out: ../6_int/confout/pos_URA3WT9_t2_conf_cell058.csv
    out: ../6_int/confout/pos_URA3WT9_t2_conf_cell059.csv
    out: ../6_int/confout/pos_URA3WT9_t2_conf_cell060.csv
    out: ../6_int/confout/pos_URA3WT9_t2_conf_cell061.csv
    out: ../6_int/confout/pos_URA3WT9_t2_conf_cell062.csv
    out: ../6_

    out: ../6_int/confout/pos_URA3WT9_t4_conf_cell008.csv
    out: ../6_int/confout/pos_URA3WT9_t4_conf_cell009.csv
    out: ../6_int/confout/pos_URA3WT9_t4_conf_cell010.csv
    out: ../6_int/confout/pos_URA3WT9_t4_conf_cell011.csv
    out: ../6_int/confout/pos_URA3WT9_t4_conf_cell012.csv
    out: ../6_int/confout/pos_URA3WT9_t4_conf_cell013.csv
    out: ../6_int/confout/pos_URA3WT9_t4_conf_cell014.csv
    out: ../6_int/confout/pos_URA3WT9_t4_conf_cell015.csv
    out: ../6_int/confout/pos_URA3WT9_t4_conf_cell016.csv
    out: ../6_int/confout/pos_URA3WT9_t4_conf_cell017.csv
    out: ../6_int/confout/pos_URA3WT9_t4_conf_cell018.csv
    out: ../6_int/confout/pos_URA3WT9_t4_conf_cell019.csv
    out: ../6_int/confout/pos_URA3WT9_t4_conf_cell020.csv
    out: ../6_int/confout/pos_URA3WT9_t4_conf_cell021.csv
    out: ../6_int/confout/pos_URA3WT9_t4_conf_cell022.csv
    out: ../6_int/confout/pos_URA3WT9_t4_conf_cell023.csv
    out: ../6_int/confout/pos_URA3WT9_t4_conf_cell024.csv
    out: ../6_

    out: ../6_int/confout/pos_URA3WT9_ta_conf_cell069.csv
    out: ../6_int/confout/pos_URA3WT9_ta_conf_cell070.csv
    out: ../6_int/confout/pos_URA3WT9_ta_conf_cell071.csv
    out: ../6_int/confout/pos_URA3WT9_ta_conf_cell072.csv
    out: ../6_int/confout/pos_URA3WT9_ta_conf_cell073.csv
    out: ../6_int/confout/pos_URA3WT9_ta_conf_cell074.csv
    out: ../6_int/confout/pos_URA3WT9_ta_conf_cell075.csv
    out: ../6_int/confout/pos_URA3WT9_ta_conf_cell076.csv
    out: ../6_int/confout/pos_URA3WT9_ta_conf_cell077.csv
    out: ../6_int/confout/pos_URA3WT9_ta_conf_cell078.csv
    out: ../6_int/confout/pos_URA3WT9_ta_conf_cell079.csv
    out: ../6_int/confout/pos_URA3WT9_ta_conf_cell080.csv
    out: ../6_int/confout/pos_URA3WT9_ta_conf_cell081.csv
    out: ../6_int/confout/pos_URA3WT9_ta_conf_cell082.csv
    out: ../6_int/confout/pos_URA3WT9_ta_conf_cell083.csv
    out: ../6_int/confout/pos_URA3WT9_ta_conf_cell084.csv
    out: ../6_int/confout/pos_URA3WT9_ta_conf_cell085.csv
    out: ../6_

In [5]:
ptabcol.head()

,strain,cond,exp,meiosis,cell,tpn,tps,com,X1,Y1,Z1,X2,Y2,Z2
0,URA3,WT,9,ta,137,1,0,pair,17,16,33,-,-,-
1,URA3,WT,9,ta,137,2,30,pair,18,16,33,-,-,-
2,URA3,WT,9,ta,137,3,60,pair,19,15,32,-,-,-
3,URA3,WT,9,ta,137,4,90,pair,19,14,32,-,-,-
4,URA3,WT,9,ta,137,5,120,pair,19,14,33,-,-,-


In [6]:
multiplefiles = glob.glob("../6_int/confout/*.csv")
multiplefiles = sorted(multiplefiles)
mcall = []
for m in multiplefiles:
    dffy = pd.read_csv(m)
    mcall.append(dffy)
mcall = pd.concat(mcall)
mcall.to_csv("../6_int/comb_xyz.csv", sep=',', index=False)
print("    out: "+"../6_int/comb_xyz.csv")

    out: ../6_int/comb_xyz.csv


In [7]:
mcall['deskc'] = mcall['strain'] +"_"+ mcall['cond'] +"_"+ mcall['exp'].apply(str) +"_"+ mcall['meiosis'] +"_c"+ mcall['cell'].apply(str)
mcall[mcall['deskc'] == 'HET5_WT_2_t0_c24'].head()

,strain,cond,exp,meiosis,cell,tpn,tps,com,X1,Y1,Z1,X2,Y2,Z2,deskc
0,HET5,WT,2,t0,24,1.0,0,NaN,n,n,n,n,n,n,HET5_WT_2_t0_c24
1,HET5,WT,2,t0,24,2.0,30,unp,24.0,21.0,23.0,17.0,15.0,27.0,HET5_WT_2_t0_c24
2,HET5,WT,2,t0,24,3.0,60,NaN,n,n,n,n,n,n,HET5_WT_2_t0_c24
3,HET5,WT,2,t0,24,4.0,90,unp,22.0,21.0,23.0,19.0,21.0,25.0,HET5_WT_2_t0_c24
4,HET5,WT,2,t0,24,5.0,120,NaN,n,n,n,n,n,n,HET5_WT_2_t0_c24


In [62]:
shyy = pd.read_csv("../6_int/comb_xyz.csv")
print("    in: "+"../6_int/comb_xyz.csv")
pozzy = shyy
pozzy['X1'] = pozzy['X1'].apply(pd.to_numeric, args=('coerce',))
pozzy['Y1'] = pozzy['Y1'].apply(pd.to_numeric, args=('coerce',))
pozzy['Z1'] = pozzy['Z1'].apply(pd.to_numeric, args=('coerce',))
pozzy['X2'] = pozzy['X2'].apply(pd.to_numeric, args=('coerce',))
pozzy['Y2'] = pozzy['Y2'].apply(pd.to_numeric, args=('coerce',))
pozzy['Z2'] = pozzy['Z2'].apply(pd.to_numeric, args=('coerce',))
pozzy['X1t'] = pozzy['X1'].isnull()
pozzy['Y1t'] = pozzy['Y1'].isnull()
pozzy['Z1t'] = pozzy['Z1'].isnull()
pozzy['X2t'] = pozzy['X2'].isnull()
pozzy['Y2t'] = pozzy['Y2'].isnull()
pozzy['Z2t'] = pozzy['Z2'].isnull()
# pozzy['X1t'] = pozzy['X1']
# pozzy['Y1t'] = pozzy['Y1']
# pozzy['Z1t'] = pozzy['Z1']
# pozzy['X2t'] = pozzy['X2']
# pozzy['Y2t'] = pozzy['Y2']
# pozzy['Z2t'] = pozzy['Z2']
yui = pozzy
yui['x1n'] = np.where(yui['X1t'] == True, np.nan, yui['X1'])
yui['y1n'] = np.where(yui['Y1t'] == True, np.nan, yui['Y1'])
yui['z1n'] = np.where(yui['Z1t'] == True, np.nan, yui['Z1'])
yui['x2n'] = np.where(yui['X2t'] == True, np.nan, yui['X2'])
yui['y2n'] = np.where(yui['Y2t'] == True, np.nan, yui['Y2'])
yui['z2n'] = np.where(yui['Z2t'] == True, np.nan, yui['Z2'])
# yui['x1n'] = yui['X1']
# yui['y1n'] = yui['Y1']
# yui['z1n'] = yui['Z1']
# yui['x2n'] = yui['X2']
# yui['y2n'] = yui['Y2']
# yui['z2n'] = yui['Z2']

    in: ../6_int/comb_xyz.csv


In [64]:
# yui['deskc'] = yui['strain'] +"_"+ yui['cond'] +"_"+ yui['exp'].apply(str) +"_"+ yui['meiosis'] +"_c"+ yui['cell'].apply(str)
# yui[yui['deskc']=='HET5_WT_2_t0_c24']

In [65]:
von = yui
von = von.drop(columns=['X1', 'Y1', 'Z1',
                       'X2', 'Y2', 'Z2', 
                       'X1t', 'Y1t', 'Z1t',
                       'X2t', 'Y2t', 'Z2t'])
von = von.rename(columns={"strain":"locus", "cond":"genotype", "exp":"exp.rep", "meiosis":"meiosis",
                         "cell":"cell", "tpn":"tp.n", "tps":"tp.s", "com":"foci",
                         "x1n":"X1", "y1n":"Y1", "z1n":"Z1",
                         "x2n":"X2", "y2n":"Y2", "z2n":"Z2"})
von = von[['locus', 'genotype', 'exp.rep', 'meiosis', 'cell',
           'tp.n', 'tp.s', 'foci', 
           'X1', 'Y1', 'Z1', 
           'X2', 'Y2', 'Z2']]
von['tp.n'] = von['tp.n'].apply(int)
von.to_csv("../6_int/comb_pos_xyz.csv", sep=',', index=False)
print("    out: "+"../6_int/comb_pos_xyz.csv")

    out: ../6_int/comb_pos_xyz.csv


In [9]:
look = von
look['deskc'] = look['locus'] +"_"+ look['genotype'] +"_"+ look['exp.rep'].apply(str) +"_"+ look['meiosis'] +"_c"+ look['cell'].apply(str)
look[look['deskc'] == 'HET5_WT_2_t0_c24'].head()

,locus,genotype,exp.rep,meiosis,cell,tp.n,tp.s,foci,X1,Y1,Z1,X2,Y2,Z2,deskc
1150,HET5,WT,2,t0,24,1,0,NaN,n,n,n,n,n,n,HET5_WT_2_t0_c24
1151,HET5,WT,2,t0,24,2,30,unp,24.0,21.0,23.0,17.0,15.0,27.0,HET5_WT_2_t0_c24
1152,HET5,WT,2,t0,24,3,60,NaN,n,n,n,n,n,n,HET5_WT_2_t0_c24
1153,HET5,WT,2,t0,24,4,90,unp,22.0,21.0,23.0,19.0,21.0,25.0,HET5_WT_2_t0_c24
1154,HET5,WT,2,t0,24,5,120,NaN,n,n,n,n,n,n,HET5_WT_2_t0_c24
